In [1]:
import os
import openai
import pandas as pd
from google.cloud import vision
import io


In [2]:

# Set up your OpenAI API key
openai.api_key = 'sk-proj-hIOossFdrZSyM5ixG4vPT3BlbkFJv9bTi0HWRujvSlWCnsIW'


In [6]:

# Set up your Google Cloud Vision API client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/njlalwani/Downloads/fiery-rarity-430115-b5-5cad704232f1.json'
client = vision.ImageAnnotatorClient()


I0000 00:00:1721680247.310922 4545997 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [10]:

def get_image_labels(image_path):
    """Use Google Vision API to get labels from the image"""
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.label_detection(image=image)
    labels = response.label_annotations

    label_descriptions = [label.description for label in labels]
    return label_descriptions


In [11]:

def generate_prompt_from_labels(labels, code, color):
    """Generate a prompt for GPT-3.5 based on image labels"""
    prompt = f"""
    Product Code: {code}
    Product Color: {color}
    Image Labels: {', '.join(labels)}
    
    Generate the following:
    - Product Title
    - Product Description
    - Keywords
    
    Please provide them in the format:
    Title: <title>
    Description: <description>
    Keywords: <keywords>
    """
    return prompt


In [12]:

def generate_product_info(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    
    # Parse the response
    response_text = response.choices[0].text.strip()
    response_lines = response_text.split('\n')
    title = response_lines[0].split(': ')[1]
    description = response_lines[1].split(': ')[1]
    keywords = response_lines[2].split(': ')[1]
    
    return title, description, keywords


In [14]:

# Directory containing your images
image_directory = '/Users/njlalwani/Library/CloudStorage/OneDrive-stevens.edu/photo_scripts/Final_photo_directory/shopify_images'
product_infos = []

# Iterate over each file in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other extensions if necessary
        name, ext = os.path.splitext(filename)
        parts = name.split('_')
        if len(parts) < 2:
            continue
        
        code = parts[0]
        color = parts[1]
        
        # Get labels from the image
        image_path = os.path.join(image_directory, filename)
        labels = get_image_labels(image_path)
        
        # Generate the prompt
        prompt = generate_prompt_from_labels(labels, code, color)
        
        # Generate product info
        title, description, keywords = generate_product_info(prompt)
        
        # Generate formatted title
        brand_name = "YourBrandName"  # Replace with your actual brand name
        product_type = "Type of Dress"  # Replace with the actual type of dress
        formatted_title = f"{brand_name} {code} {product_type} in {color}"
        
        product_infos.append({
            'Code': code,
            'Color': color,
            'Product Title': formatted_title,
            'Product Description': description,
            'Keywords': keywords
        })

# Save the data to a CSV file
df = pd.DataFrame(product_infos)
df.to_csv('product_info.csv', index=False)
print("Product information has been saved to product_info.csv")


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [7]:
import os
import openai
import pandas as pd
from google.cloud import vision
import io


def get_image_labels(image_path):
    """Use Google Vision API to get labels from the image"""
    try:
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.label_detection(image=image)
        labels = response.label_annotations

        label_descriptions = [label.description for label in labels]
        return label_descriptions
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return []

def generate_prompt_from_labels(labels, code, color):
    """Generate a prompt for GPT-3.5 based on image labels"""
    prompt = f"""
    Product Code: {code}
    Product Color: {color}
    Image Labels: {', '.join(labels)}
    
    Generate the following:
    - Product Title
    - Product Description
    - Keywords
    
    Please provide them in the format:
    Title: <title>
    Description: <description>
    Keywords: <keywords>
    """
    return prompt

def generate_product_info(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    
    # Parse the response
    response_text = response.choices[0].message['content'].strip()
    response_lines = response_text.split('\n')
    title = response_lines[0].split(': ')[1]
    description = response_lines[1].split(': ')[1]
    keywords = response_lines[2].split(': ')[1]
    
    return title, description, keywords

# Directory containing your images
image_directory = '/Users/njlalwani/Library/CloudStorage/OneDrive-stevens.edu/photo_scripts/Final_photo_directory/shopify_images'
product_infos = []

# Iterate over each file in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other extensions if necessary
        name, ext = os.path.splitext(filename)
        parts = name.split('_')
        if len(parts) < 2:
            continue
        
        code = parts[0]
        color = parts[1]
        
        # Get labels from the image
        image_path = os.path.join(image_directory, filename)
        labels = get_image_labels(image_path)
        
        if not labels:
            print(f"Skipping image {filename} due to Vision API error.")
            continue
        
        # Generate the prompt
        prompt = generate_prompt_from_labels(labels, code, color)
        
        # Generate product info
        title, description, keywords = generate_product_info(prompt)
        
        # Generate formatted title
        brand_name = "YourBrandName"  # Replace with your actual brand name
        product_type = "Type of Dress"  # Replace with the actual type of dress
        formatted_title = f"{brand_name} {code} {product_type} in {color}"
        
        product_infos.append({
            'Code': code,
            'Color': color,
            'Product Title': formatted_title,
            'Product Description': description,
            'Keywords': keywords
        })

# Save the data to a CSV file
df = pd.DataFrame(product_infos)
df.to_csv('product_info.csv', index=False)
print("Product information has been saved to product_info.csv")



IndexError: list index out of range

In [8]:
import os
import openai
import pandas as pd
from google.cloud import vision
import io


def get_image_labels(image_path):
    """Use Google Vision API to get labels from the image"""
    try:
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.label_detection(image=image)
        labels = response.label_annotations

        label_descriptions = [label.description for label in labels]
        return label_descriptions
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return []

def generate_prompt_from_labels(labels, code, color):
    """Generate a prompt for GPT-3.5 based on image labels"""
    prompt = f"""
    Product Code: {code}
    Product Color: {color}
    Image Labels: {', '.join(labels)}
    
    Generate the following:
    - Product Title
    - Product Description
    - Keywords
    
    Please provide them in the format:
    Title: <title>
    Description: <description>
    Keywords: <keywords>
    """
    return prompt

def generate_product_info(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )
        
        # Parse the response
        response_text = response.choices[0].message['content'].strip()
        response_lines = response_text.split('\n')
        
        title = response_lines[0].split(': ')[1] if len(response_lines) > 0 else 'N/A'
        description = response_lines[1].split(': ')[1] if len(response_lines) > 1 else 'N/A'
        keywords = response_lines[2].split(': ')[1] if len(response_lines) > 2 else 'N/A'
        
        return title, description, keywords
    except Exception as e:
        print(f"Error generating product info: {e}")
        return 'N/A', 'N/A', 'N/A'

product_infos = []

# Iterate over each file in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other extensions if necessary
        name, ext = os.path.splitext(filename)
        parts = name.split('_')
        if len(parts) < 2:
            continue
        
        code = parts[0]
        color = parts[1]
        
        # Get labels from the image
        image_path = os.path.join(image_directory, filename)
        labels = get_image_labels(image_path)
        
        if not labels:
            print(f"Skipping image {filename} due to Vision API error.")
            continue
        
        # Generate the prompt
        prompt = generate_prompt_from_labels(labels, code, color)
        
        # Generate product info
        title, description, keywords = generate_product_info(prompt)
        
        # Generate formatted title
        brand_name = "YourBrandName"  # Replace with your actual brand name
        product_type = "Type of Dress"  # Replace with the actual type of dress
        formatted_title = f"{brand_name} {code} {product_type} in {color}"
        
        product_infos.append({
            'Code': code,
            'Color': color,
            'Product Title': formatted_title,
            'Product Description': description,
            'Keywords': keywords
        })

# Save the data to a CSV file
df = pd.DataFrame(product_infos)
df.to_csv('product_info.csv', index=False)
print("Product information has been saved to product_info.csv")


Error generating product info: list index out of range
Error generating product info: list index out of range
Error generating product info: list index out of range
Error generating product info: list index out of range
Error generating product info: list index out of range
Error generating product info: list index out of range
Error generating product info: list index out of range


KeyboardInterrupt: 